In [10]:
#Load packages
using DataFrames, CSV, Statistics, Distributions, GLM, Plots, Random, StatsModels, StatFiles, FixedEffectModels


In [12]:
# Load DataFrames
df = DataFrame(load("fulton.dta"));

## Problem 1
On the Fulton fish market about 35 different wholesale dealers sell fish to the customers that visit the market from Monday to Friday. There are no posted prices and dealers can (and do) charge different prices to different consumers. In this exercise you will analyze data which were recovered from a single dealer on the Fulton fish market. The data run from December 2nd, 1991 through May 8th, 1992 and record daily transactions in whiting. Whiting is a cheap fish which is oily and distinctive tasting. Whiting vary relatively little in size and quality, and there is probably little scope for substitution with other types of fish. Use fulton.dta. The dataset contains the quantity of whiting sold in pounds (qty) and the average (quantity weighted) prices in dollars per pound (price), information about the day of the week (Monday-Friday) and the weather conditions (stormy, mixed)

1.Write a two-equation system in “supply and demand form”, that is, with the same
variable (typically, quantity) appearing on the left-hand side:
$$
qty = \alpha_1 price + \beta_1 z_1 + u_1 \tag{1}
$$
$$
qty = \alpha_2 price + \beta_2 z_2 + u_2 \tag{2}
$$
If $\alpha_1 = 0$ or $\alpha_2 = 0$, explain why a reduced form exists for $qty$. (Remember, a reduced form expresses $qty$ as a linear function of the exogenous variables and the structural errors.) If $\alpha_1 \ne0$ and $\alpha_2 = 0$, find the reduced form for price.

Without loss of generality, we can assume that $\alpha_2=0$ and $\alpha_1 \ne 0$ and then the functions can be reduced to
$$
qty = \beta_2 z_2 + u_2
$$
Which is a reduced form for $qty$.


Then we can equate the two equations to have:
$$
\alpha_1 price + \beta_1 z_1 + u_1 = \beta_2 z_2 + u_2 
$$
And thus 
$$
price = \frac{\beta_2 z_2 + u_2-\beta_1 z_1 - u_1}{\alpha_1}
$$

2.If $\alpha_1 \ne 0$, $\alpha_2 \ne 0$ and $\alpha_1 \ne \alpha_2$, find the reduced for for $qty$. Does price have a reduced form in this case?

From equation (1), we can rewrite it as a function of $price$:
$$
price = \frac{qty-\beta_1 z_1 - u_1}{\alpha_1} \tag{3}
$$
Then plug it into equation (2):
$$
\begin{align*}
qty &= \alpha_2 \frac{qty-\beta_1 z_1 - u_1}{\alpha_1} + \beta_2 z_2 +u_2 \\
&= \frac{\alpha_2}{\alpha_1}qty-\alpha_2 \frac{\beta_1 z_1 + u_1}{\alpha_1}+\beta_2 z_2 +u_2\\
&= \frac{-\alpha_2 \frac{\beta_1 z_1 + u_1}{\alpha_1}+\beta_2 z_2 +u_2}{1-\frac{\alpha_2}{\alpha_1}} \tag{4}
\end{align*}
$$
Which is the reduced form for $qty$.

We then plug equation (2) into equation (3):
$$
\begin{align*}
price = \frac{\alpha_2 price + \beta_2 z_2 + u_2-\beta_1 z_1 - u_1}{\alpha_1}
\end{align*}
$$
Isolate $price$ to have:
$$
\begin{align*}
price &= \frac{\frac{\beta_2 z_2 + u_2-\beta_1 z_1 - u_1}{\alpha_1}}{1-\frac{\alpha_2}{\alpha_1}}\\
&= \frac{\beta_2 z_2 + u_2-\beta_1 z_1 - u_1}{\alpha_1-\alpha_2} \tag{5}
\end{align*}
$$
Which is the reduced form for $price$.

3.Is the condition $\alpha_1 \ne \alpha_2$ likely to be met in supply and demand examples? Explain.

Yes, in reality it is very likely that the two coefficients are not the same. The reason is that the coefficient for price on quantity represents the elasticity, and that for demand and supply comes from very different sources. In the case of fish, the demand elasticity can be determined by factors like the preference for fish and other substitutes like meat and the supply elasticity can be determined by the operation and technology of the fishing industry.

4.The following simultaneous equations model imposes the equilibrium condition that supply equals demand:
$$
qty = \alpha_1 price +\beta_1 stormy + u_1 \tag{6}
$$
$$
qty = \alpha_2 price +\beta_2 friday + u_2 \tag{7}
$$
Which would you argue is the supply equation and which is the demand equation?
Explain. In what follows suppose $\beta_2 = 0$.

Equation (6) should be the supply equation because when weather is stormy, it should be become hard to catch fish, reducing the supply. Equation (7) is the demand function because people might want more fish on Friday for some reason. 

$\beta_2 = 0$ means that people's consumption of fish is the same on Friday compared to other days in a week.

5.Do the supply and demand equations satisfy the order condition for identification?

Since we have two exogenous variables: stormy and friday, and also two endogenous variables, so $H=K$ which satisifies the condition for exact identification.

7.Estimate reduced form equations for qty and price. Comment on the reduced form
coefficients on stormy

Replace $z_1$ and $z_2$ with $stormy$ and $friday$ from the reduced form equations (4) and (5) and isolate the exogenous variables on the right hand sides to have:
$$
qty = -\frac{\alpha_2 \beta_1}{\alpha_1 - \alpha_2}stormy +\frac{\alpha_1 \beta_2}{\alpha_1 - \alpha_2}friday + u_{qty} \tag{8}
$$
$$
price = \frac{\beta_2}{\alpha_1-\alpha_2}friday - \frac{\beta_1}{\alpha_1 - \alpha_2}stormy + u_{price} \tag{9}
$$

In [6]:
# Modify the data
df.stormy = Float64.(df.stormy)
df.friday = Float64.(df.friday)
df.qty = Float64.(df.qty)

# Estimate the supply function
model_qty = lm(@formula(qty ~ stormy + friday),df)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

qty ~ 1 + stormy + friday

Coefficients:
──────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error      t  Pr(>|t|)  Lower 95%   Upper 95%
──────────────────────────────────────────────────────────────────────────
(Intercept)   8.58036    0.0892642  96.12    <1e-99   8.40342    8.7573
stormy       -0.356755   0.151787   -2.35    0.0206  -0.657622  -0.0558871
friday        0.221614   0.169636    1.31    0.1942  -0.114633   0.557861
──────────────────────────────────────────────────────────────────────────

For the stormy dummy variable, the coefficient is negative which matches the idea that during a stormy day the fish supply is decreased and it is also statistically significant at 5% level.

For the friday dummy variable, the coefficient is positive which means more fish are supplied on Friday for some reason but it is not statistically significant so we cannot reject the null hypothesis that fish supply on Friday is not different from other days.

In [7]:
# Modify the data
df.price = Float64.(df.price)
# Estimate the demand function
model_price = lm(@formula(price ~ stormy + friday),df)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

price ~ 1 + stormy + friday

Coefficients:
──────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────
(Intercept)  -0.296184    0.0435453  -6.80    <1e-09  -0.382499  -0.20987
stormy        0.336015    0.0740453   4.54    <1e-04   0.189245   0.482786
friday        0.0271893   0.0827525   0.33    0.7431  -0.13684    0.191219
──────────────────────────────────────────────────────────────────────────

For the stormy dummy variable, the coefficient in positive and statistically significant, meaning that the price goes up on stormy day, which match the fact that the suply on stormy day will decrease.
And the Friday dummy variable is positive but not statistically significant, meaning we cannot reject the null hypothesis that price on friday is different from the others.

7.Divide the reduced form coefficient on stormy in the reduced form quantity equation
by the reduced form coefficient on stormy in the reduced form price equation. What
parameter are you estimating now? Interpret on the sign and size of this estimate.

From equation (8) and (9) we have:
$$
\frac{-\frac{\alpha_2 \beta_1}{\alpha_1 - \alpha_2}}{- \frac{\beta_1}{\alpha_1 - \alpha_2}}=\alpha_2
$$
which is exactly the coefficient on price in the demand curve. The reason behind is that stormy shift the supply curve which help us identify the demand curve.


Numerically: $-0.356755/0.336015=-1.0617$

The sign is negative, meaning that when price increases, quantity demand decreases, matching the normal demand curve. And $1$ dollar per pound increase in price decreases the quantity demand by $-1.0617$ pounds.

8.Estimate the same parameter in (7) using 2SLS (-ivreg- in Stata)

In [11]:
# So we essentially treat stormy as a instrument for price.
# Estimate the 2SLS model
demand_2sls= reg(df, @formula(qty ~ (price ~ stormy)))

                             FixedEffectModel                             
Number of obs:                   111   Converged:                      true
dof (model):                       1   dof (residuals):                 108
R²:                           -0.000   R² adjusted:                  -0.009
F-statistic:                 5.40174   P-value:                       0.022
F-statistic (first stage):   21.0696   P-value (first stage):         0.000
             Estimate  Std. Error    t-stat  Pr(>|t|)  Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
price        -1.08241    0.46572   -2.32416    0.0220   -2.00555  -0.159272
(Intercept)   8.31379    0.114622  72.5319     <1e-92    8.08659   8.54099


9.Can you find an explanation why we assumed that β2 = 0 (Hint: include it in the
reduced form estimations)

In the reduced form estimation we have the coefficients on Friday are given by:
$$
\frac{\alpha_1 \beta_2}{\alpha_1 - \alpha_2}\\
\frac{\beta_2}{\alpha_1 - \alpha_2}
$$
Since in the reduced form estimation we found that both coefficients are not significant, so it is safe to assume them to be $0$. To achieve this, we can assume $\beta_2 = 0$.